## Graphs

### Minimum cuts:
    In a graph, cluster the vertices into 2 groups such that the edges crossing from 1 cluste to another cluster are minimum.

#### Random contraction algorithm

        Contraction: Contraction of a edge is combining the 2 vertexes of that edge in to one vertex and removing any self loops.

        Pick a random edge from the graph G and contract this edge. Do this until there is only one edge left. When there is only once edge left, i.e., the vertexes have been clustered into two groups, consider that these clusters result in the minimum cut of the graph G.
        Note: The problem is, since the edges are chosen at random, the probablity of success is not very high(the mathmatical analysis in book). But doing this experiment repeated number of times will result in minimum cut with a high probablity.
        
    
        

In [193]:
class Vertex:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

class Edge:
    def __init__(self, source_vertex : Vertex, destination_vertex:Vertex):
        self.source_vertex = source_vertex
        self.destination_vertex = destination_vertex
        self.name = source_vertex.name + '-' + destination_vertex.name
    
    def __str__(self):
        return f"{self.source_vertex} <--> {self.destination_vertex}"
    
    def __repr__(self):
        return f"{self.source_vertex} <--> {self.destination_vertex}"

class Graph:

    def __init__(self, name: str):
        self.name = name
        self.vertexes = {}
        self.edges = {}
        self.all_edges = {}

    def add_vertex(self, vertex:str):
        V = Vertex(vertex)
        if vertex in self.vertexes.keys():
            raise Exception(f"Vertex '{vertex}' is already a part of the graph")
        self.vertexes[vertex]=V
    
    def list_vertexes(self):
        print(f"verteces: {list(self.vertexes.keys())}")

    def drop_vertex(self, vertex:str):
        if vertex not in self.vertexes.keys():
            raise Exception(f"Vertex '{vertex}' not found")
        self.vertexes.pop(vertex)

    def add_edge(self, source_vertex: str , destination_vertex: str):

        E = Edge(
            self.vertexes[source_vertex], 
            self.vertexes[destination_vertex]
        )
        if E.name in self.all_edges.keys():
            raise ValueError(f"Edge '{E}' is already a part of the graph")
        
        self.edges[E.name]=E
        self.all_edges[E.name]=E
        other_edge = Edge(
            self.vertexes[destination_vertex],
            self.vertexes[source_vertex],
        )
        self.all_edges[other_edge.name] = other_edge
    
    def list_edges(self):
        print(f"edges: {list(self.edges.keys())}")

    def drop_edge(self, edge_name):
        try: 
            E = self.edges[edge_name]
            edge_name = E.source_vertex.name + "-" + E.destination_vertex.name
            other_edge_name = E.destination_vertex.name + "-" + E.source_vertex.name
            # remove the edge from normal list
            self.edges.pop(edge_name)
            # remove the edges from all edges list
            self.all_edges.pop(edge_name)
            self.all_edges.pop(other_edge_name)
        except KeyError as e:
            raise Exception(f"Edge {edge_name} not found")

    def contract(edge):
        pass

    def __str__(self) -> str:
        return self.name
    
    def __repr__(self):
        return f"{self.name}"
    

In [194]:
# create a graph with lists of vertexes and nodes
def create_graph(graph_name,vertexes, edges):
    G = Graph(graph_name)
    for vertex in vertexes.split(','):
        G.add_vertex(vertex)
    for edge in edges.split(','):
        try:
            G.add_edge(edge.split('-')[0], edge.split('-')[1])
        except ValueError as e:
            print(e)
    
    G.list_vertexes()
    G.list_edges()
    print("")
    return G

# nodes = "abcdefgh"
# edges = "ab,ae,be,bc,cd,de,ef,eg,fg,gh"
# G = create_graph(nodes,edges)
    


In [195]:
def contract(G: Graph, e: Edge):
    combined_vertex_name = (e.source_vertex.name +","+ e.destination_vertex.name)

    # add the new vertex to the graph
    G.add_vertex(combined_vertex_name)

    # remove the vertexes that correspond to the contracting edge
    G.drop_vertex(e.source_vertex.name)
    G.drop_vertex(e.destination_vertex.name)

    # remove the edge that needs to be contracted
    G.drop_edge(e.name)

    # rearrange the edges that have nodes correspoding to the contracted edges
    for key in list(G.edges.keys())[:]:
        if G.edges[key].source_vertex.name in [e.source_vertex.name, e.destination_vertex.name]:
            try:
                G.add_edge(
                    combined_vertex_name, 
                    G.edges[key].destination_vertex.name
                )
            except ValueError as E:
                pass
            G.drop_edge(key)
        elif G.edges[key].destination_vertex.name in [e.source_vertex.name, e.destination_vertex.name]:
            try:
                G.add_edge(
                    G.edges[key].source_vertex.name, 
                    combined_vertex_name
                )
            except ValueError as E:
                pass
            G.drop_edge(key)
    return G
    
# nodes = "a,b,c,d,e,f,g,h"
# edges = "a-b,a-e,b-e,b-c,c-d,d-e,e-f,e-g,f-g,g-h"
# G = create_graph('G',nodes,edges)

# G = contract(G, G.edges['b-e'])
# G.edges

In [196]:
from copy import deepcopy
from random import Random
def random_contraction  (G: Graph, verbose=False):
    rand = Random()
    G_t = deepcopy(G)
    
    # contract the vertex of the graph, one vertex at a time
    if verbose : print("""-------- Ruuning contraction Algorithm --------""")
    while len(G_t.edges) >1:
        edge_to_contract = rand.choice(list(G_t.edges.keys()))
        if verbose : print(f"Contracting edge: {edge_to_contract}")
        G_t = contract(G_t, G_t.edges[edge_to_contract])
        if verbose : print(f"Graph: {list(G_t.edges.keys())}")
        if verbose : print("")
    if verbose : print("""-----------------------------------------------""",end="\n\n")

    print(f"Contracted Graph : {G_t.edges}",end="\n")

    # count the number of edges between contracted vertexes
    assert len(G_t.vertexes) == 2, "contraction algorithm resulted in more than or less than 2 vertexes"

    grp1, grp2 = list(G_t.vertexes.values())[0].name,list(G_t.vertexes.values())[1].name
    edge_count=0
    for ver1 in grp1.split(','):
        for ver2 in grp2.split(','):
            for edge in G.all_edges:
                if ver1 == G.all_edges[edge].source_vertex.name and ver2 == G.all_edges[edge].destination_vertex.name:
                    if verbose: print(f"counting edge: {edge}")
                    edge_count+=1
    print(f"Minimum edge count : {edge_count}")
    return G_t, edge_count


# nodes = "a,b,c,d,e,f,g,h"
# edges = "a-b,a-e,b-e,b-c,c-d,d-e,e-f,e-g,f-g,g-h"
# G = create_graph('G',nodes,edges)
# contracted_G = random_contraction(G)



In [201]:
# repeated runs of random contraction algorithm to get minimum cut of the graph

def random_contraction_algorithm(G: Graph):
    minimum_cuts_tracker = {}
    minimum_cuts = None
    minimum_cut_edge = None
    for _ in range(len(G.vertexes)*2):
        print(f"\n run_{_}")
        contracted_G, edge_count = random_contraction(G)
        print(f"contracted_edges : {contracted_G.edges}, no of cuts = {edge_count}")
        minimum_cuts_tracker[list(contracted_G.edges.keys())[0]] = edge_count
        if minimum_cuts == None or edge_count < minimum_cuts:
            minimum_cuts = edge_count
            minimum_cut_edge = contracted_G.edges
    print("##################################################")
    print(f"minimum_cut_edge : {contracted_G.edges}")
    print(f"minimum_cuts: {minimum_cuts}")
    return minimum_cut_edge, minimum_cuts

nodes = "a,b,c,d,e,f,g,h"
edges = "a-b,a-e,b-e,b-c,c-d,d-e,e-f,e-g,f-g,g-h"
G = create_graph('G',nodes,edges)
minimum_cut_edge, minimum_cuts = random_contraction_algorithm(G)

verteces: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
edges: ['a-b', 'a-e', 'b-e', 'b-c', 'c-d', 'd-e', 'e-f', 'e-g', 'f-g', 'g-h']


 run_0
Contracted Graph : {'c,d,a,b,e,f,g-h': c,d,a,b,e,f,g <--> h}
Minimum edge count : 1
contracted_edges : {'c,d,a,b,e,f,g-h': c,d,a,b,e,f,g <--> h}, no of cuts = 1

 run_1
Contracted Graph : {'a,b,c-d,e,f,g,h': a,b,c <--> d,e,f,g,h}
Minimum edge count : 3
contracted_edges : {'a,b,c-d,e,f,g,h': a,b,c <--> d,e,f,g,h}, no of cuts = 3

 run_2
Contracted Graph : {'a,b,e,f,g,c,d-h': a,b,e,f,g,c,d <--> h}
Minimum edge count : 1
contracted_edges : {'a,b,e,f,g,c,d-h': a,b,e,f,g,c,d <--> h}, no of cuts = 1

 run_3
Contracted Graph : {'c,a,b,d,e,f-g,h': c,a,b,d,e,f <--> g,h}
Minimum edge count : 2
contracted_edges : {'c,a,b,d,e,f-g,h': c,a,b,d,e,f <--> g,h}, no of cuts = 2

 run_4
Contracted Graph : {'a,b,c,d-e,f,g,h': a,b,c,d <--> e,f,g,h}
Minimum edge count : 3
contracted_edges : {'a,b,c,d-e,f,g,h': a,b,c,d <--> e,f,g,h}, no of cuts = 3

 run_5
Contracted Graph 

In [202]:
# Assignment

all_vertexes = []
all_edges = []
with open('kargerMinCut.txt', 'r') as file:
    for line in file:
        line = line.split()
        vertex = line[0]
        for adj_vertex in line[1:]:
            all_edges.append(f"{vertex}-{adj_vertex}")
        all_vertexes.append(vertex)

all_vertexes = ",".join(all_vertexes)
all_edges = ",".join(all_edges)
G = create_graph('G', all_vertexes, all_edges)

minimum_cut_edge, minimum_cuts = random_contraction_algorithm(G)

Edge '3 <--> 2' is already a part of the graph
Edge '4 <--> 1' is already a part of the graph
Edge '8 <--> 7' is already a part of the graph
Edge '9 <--> 8' is already a part of the graph
Edge '10 <--> 2' is already a part of the graph
Edge '12 <--> 2' is already a part of the graph
Edge '12 <--> 11' is already a part of the graph
Edge '13 <--> 2' is already a part of the graph
Edge '13 <--> 10' is already a part of the graph
Edge '13 <--> 11' is already a part of the graph
Edge '14 <--> 6' is already a part of the graph
Edge '15 <--> 1' is already a part of the graph
Edge '15 <--> 7' is already a part of the graph
Edge '16 <--> 10' is already a part of the graph
Edge '16 <--> 3' is already a part of the graph
Edge '17 <--> 3' is already a part of the graph
Edge '18 <--> 15' is already a part of the graph
Edge '18 <--> 1' is already a part of the graph
Edge '18 <--> 4' is already a part of the graph
Edge '19 <--> 5' is already a part of the graph
Edge '19 <--> 14' is already a part of 

In [200]:
minimum_cuts

1